In [7]:
## Pipeline com Catboost

In [ ]:
import pandas as pd
import numpy as np
import optuna

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from scipy import stats
from catboost import CatBoostClassifier

# --- Funções e Configurações Iniciais ---

def calcular_ks_scipy(y_true: np.ndarray, y_pred_proba: np.ndarray) -> float:
    """Calcula a estatística KS (Kolmogorov-Smirnov)."""
    prob_bads = y_pred_proba[y_true == 1]
    prob_goods = y_pred_proba[y_true == 0]
    if len(prob_bads) == 0 or len(prob_goods) == 0:
        return 0.0
    ks_statistic, _ = stats.ks_2samp(prob_bads, prob_goods)
    return ks_statistic

# --- Carregamento dos Dados ---
try:
    treino_df = pd.read_csv("../data/dev/train.csv", index_col=0)
    validacao_df = pd.read_csv("../data/dev/val.csv", index_col=0)
    teste_oot_df = pd.read_csv("../data/dev/test.csv", index_col=0)

    dev_df = pd.concat([treino_df, validacao_df], ignore_index=True)
    
    TARGET = 'target'
    FEATURES = [col for col in dev_df.columns if col not in [
        TARGET, 'data_originacao', 'id_contrato', 'custo_fn', 'custo_fp'
    ]]

    X_dev = dev_df[FEATURES]
    y_dev = dev_df[TARGET]
    
    X_oot = teste_oot_df[FEATURES]
    y_oot = teste_oot_df[TARGET]

except FileNotFoundError as e:
    print(f"Problema com os dados: {e}. Verifique os caminhos dos arquivos.")
    exit()

# --- Identificação de Features Categóricas para o CatBoost ---
categorical_features_indices = [i for i, col in enumerate(X_dev.columns) if X_dev[col].dtype == 'object']
print(f"Features categóricas identificadas: {[X_dev.columns[i] for i in categorical_features_indices]}")
print("\n" + "="*80 + "\n")


# --- 1. DEFINIÇÃO DA FUNÇÃO OBJECTIVE PARA O OPTUNA ---

def objective(trial: optuna.Trial) -> float:
    """
    Função que o Optuna tentará otimizar.
    Ela treina um modelo CatBoost com os hiperparâmetros sugeridos pelo 'trial'
    e retorna o AUC médio da validação cruzada temporal.
    """
    # Espaço de busca dos hiperparâmetros
    params = {
        'objective': 'Logloss',
        'eval_metric': 'AUC',
        'iterations': 1500,  # Aumentamos um pouco o teto, pois o early stopping vai agir
        'verbose': 0,
        'random_seed': 42,
        'cat_features': categorical_features_indices,
        'allow_writing_files': False, # Desativa a criação de arquivos de log do catboost

        # Hiperparâmetros a serem otimizados pelo Optuna
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 30.0, log=True),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
    }
    
    # Ajustes condicionais para o bootstrap_type 'Bernoulli'
    if params['bootstrap_type'] == 'Bernoulli':
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1.0)

    tscv = TimeSeriesSplit(n_splits=5)
    fold_auc_scores = []

    for train_index, val_index in tscv.split(X_dev):
        X_train_fold, X_val_fold = X_dev.iloc[train_index], X_dev.iloc[val_index]
        y_train_fold, y_val_fold = y_dev.iloc[train_index], y_dev.iloc[val_index]

        model = CatBoostClassifier(**params)
        
        model.fit(
            X_train_fold, y_train_fold,
            eval_set=(X_val_fold, y_val_fold),
            early_stopping_rounds=50,
            verbose=False
        )
        
        preds = model.predict_proba(X_val_fold)[:, 1]
        auc_fold = roc_auc_score(y_val_fold, preds)
        fold_auc_scores.append(auc_fold)

    return np.mean(fold_auc_scores)

# --- 2. EXECUÇÃO DO ESTUDO DE OTIMIZAÇÃO ---

print("---" * 10 + " INICIANDO O TUNING DE HIPERPARÂMETROS COM OPTUNA " + "---" * 10)

# Criamos um "estudo", que gerencia o processo de otimização
study = optuna.create_study(direction='maximize')

# Iniciamos a otimização. O Optuna chamará a função 'objective' 100 vezes.
study.optimize(objective, n_trials=100, show_progress_bar=True)

print("\nOtimização concluída!")
print(f"Melhor valor de AUC na validação cruzada: {study.best_value:.4f}")
print("Melhores hiperparâmetros encontrados:")
print(study.best_params)
print("\n" + "="*80 + "\n")


# --- 3. TREINAMENTO DO MODELO FINAL COM OS MELHORES PARÂMETROS ---

print("---" * 10 + " TREINANDO MODELO FINAL OTIMIZADO E AVALIANDO NO OOT " + "---" * 10)

# Pegando os melhores parâmetros encontrados
best_params = study.best_params
# Adicionando parâmetros fixos
best_params.update({
    'iterations': 2000, # Aumentamos um pouco para o treino final
    'verbose': 200,
    'random_seed': 42,
    'cat_features': categorical_features_indices,
    'objective': 'Logloss',
    'eval_metric': 'AUC',
})


final_model_tuned = CatBoostClassifier(**best_params)

# Treinando com o dataset de desenvolvimento completo (treino + validação)
final_model_tuned.fit(X_dev, y_dev)


# --- 4. AVALIAÇÃO FINAL ---

y_oot_pred_proba_tuned = final_model_tuned.predict_proba(X_oot)[:, 1]

auc_oot_tuned = roc_auc_score(y_oot, y_oot_pred_proba_tuned)
ks_oot_tuned = calcular_ks_scipy(y_oot.values, y_oot_pred_proba_tuned)

print("\n" + "---" * 10 + " RESULTADO FINAL (OOT) - CatBoost Otimizado " + "---" * 10)
print(f">> Performance no Teste Out-of-Time: AUC = {auc_oot_tuned:.4f}, KS = {ks_oot_tuned:.4f}")
print("---" * 27)

print("\nComparativo com o modelo anterior (não otimizado):")
print("AUC OOT (sem tuning): 0.6579")
print("KS OOT (sem tuning):  0.2141")

[I 2025-06-26 10:12:12,583] A new study created in memory with name: no-name-306af55b-1c48-4cec-a130-ffefaf665c34


Features categóricas identificadas: ['tipo_cliente', 'estado', 'qtd_contratos_anteriores_bin', 'total_parcelas_historicas_bin', 'qtd_atrasos_historicos_bin', 'qtd_antecipacoes_historicas_bin', 'taxa_atraso_historica_bin', 'taxa_antecipacao_historica_bin', 'max_atraso_dias_historico_bin', 'media_valor_financiado_anterior_bin', 'max_valor_financiado_anterior_bin', 'media_prazo_anterior_bin', 'max_prazo_anterior_bin', 'dias_desde_ultimo_contrato_bin', 'dias_como_cliente_bin', 'qtd_parcelas_em_aberto_bin', 'valor_total_em_aberto_bin']


------------------------------ INICIANDO O TUNING DE HIPERPARÂMETROS COM OPTUNA ------------------------------


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-06-26 10:13:00,434] Trial 0 finished with value: 0.6427088482981062 and parameters: {'learning_rate': 0.02214216809047802, 'depth': 5, 'l2_leaf_reg': 1.2818031884122922, 'colsample_bylevel': 0.5682611095761476, 'min_child_samples': 50, 'bootstrap_type': 'Bayesian'}. Best is trial 0 with value: 0.6427088482981062.
[I 2025-06-26 10:13:50,176] Trial 1 finished with value: 0.6329256031315174 and parameters: {'learning_rate': 0.16768495632456715, 'depth': 10, 'l2_leaf_reg': 6.35959507695476, 'colsample_bylevel': 0.9958648687181803, 'min_child_samples': 75, 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.6427088482981062.
[I 2025-06-26 10:14:13,723] Trial 2 finished with value: 0.6374600264628288 and parameters: {'learning_rate': 0.12182818459673328, 'depth': 7, 'l2_leaf_reg': 1.7008020208218673, 'colsample_bylevel': 0.48903681706352997, 'min_child_samples': 37, 'bootstrap_type': 'Bernoulli', 'subsample': 0.6039832627731663}. Best is trial 0 with value: 0.6427088482981062.
[I